<a href="https://colab.research.google.com/github/justin95214/logistics_colab_code/blob/main/(MFC)COG_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COG 알고리즘 패키지 다운로드

In [ ]:
#!git clone https://github.com/justin95214/logistic

Cloning into 'logistic'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 215 (delta 97), reused 132 (delta 83), pack-reused 57
Receiving objects: 100% (215/215), 1.61 MiB | 6.82 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [ ]:
#%cd /content/logistic

[Errno 2] No such file or directory: '/content/logistic'
/content


In [ ]:
#from cog import Cog

from google.colab import files
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame
import io
import os
import warnings
warnings.filterwarnings("ignore")
"""
from cog import CogCog: 이 줄은 라는 모듈에서 클래스를 가져오려고 시도합니다 cog. 그러나 해당 cog모듈은 표준 라이브러리나 널리 알려진 모듈이 아니므로 추가 정보 없이는 기능을 확인할 수 없습니다.

from google.colab import files: 이 줄은 패키지 files에서 모듈을 가져옵니다 google.colab. Google Colab 환경에서 파일을 업로드하고 다운로드할 수 있는 기능을 제공합니다.

import numpy as np: 이 줄은 NumPy 라이브러리를 가져오고 별칭을 할당합니다 np. NumPy는 대규모 다차원 배열 및 수학 함수를 지원하는 Python의 수치 계산을 위한 인기 있는 라이브러리입니다.

import pandas as pd: 이 줄은 Pandas 라이브러리를 가져오고 별칭을 할당합니다 pd. Pandas는 효율적인 데이터 분석 및 조작을 위한 데이터 구조 및 기능을 제공하는 강력한 데이터 조작 라이브러리입니다.

from pandas.core.frame import DataFrame: 이 줄은 모듈 DataFrame에서 클래스를 가져옵니다 pandas.core.frame. 클래스는 DataFrame레이블이 지정된 축(행 및 열)이 있는 2차원 테이블 형식 데이터 구조를 나타내는 Pandas의 중심 데이터 구조입니다.

import io: 이 줄은 io입력 및 출력 작업 작업을 위한 클래스와 함수를 제공하는 모듈을 가져옵니다.

import os: 이 줄은 os파일 및 디렉터리 액세스와 같이 운영 체제와 상호 작용하는 방법을 제공하는 모듈을 가져옵니다.

import warnings: 이 줄은 warnings파이썬이 발행한 경고 메시지를 제어할 수 있는 모듈을 가져옵니다. 특정 경고 유형을 필터링하고 무시하는 기능을 제공합니다.

이 warnings.filterwarnings("ignore")줄은 코드 실행에서 경고 메시지를 억제하는 데 사용됩니다. 이는 출력량을 줄이거나 코드 기능에 중요하지 않은 특정 경고 메시지를 숨기는 데 유용할 수 있습니다."""

In [ ]:
class Cog:

    #초기화
    def __init__(self, file_url, fixed_point, dynamic_count):
        self.file = file_url
        self.fixed_point = fixed_point
        self.name_list = ['거점'+str(i+1) for i in range(len(fixed_point))]


    # 엑셀 데이터 로드
    def load_excel_data(self):

        data = pd.read_excel(self.file)
        return data

    # 곡률값 계산 및 정렬
    def cal_curvature_value(self, df):

        df['경도 곡률값'] = df['경도\n(Longitude)'] * 3600
        df['위도 곡률값'] = df['위도\n(Latitude)'] * 3600
        df = df.sort_values(by='Demand\n(Vi)', ascending=False)
        df = df.reset_index()
        return df


    # 고정된 거점 곡률값과 정한 거점 곡률값 리스트 생성
    # 동적 정할 거점 개수 = count
    def make_position(self, df, count):
        point_list = []
        position_list = [  (df.loc[i, '위도 곡률값'], df.loc[i, '경도 곡률값'])  for i in range(count) ]
        point_list.extend(self.fixed_point)
        point_list.extend(position_list)
        
        return point_list


    def classified_position(self, df , point_list):

        for i in range(len(point_list)):
            df['거점'+ str(i+1)] = np.sqrt((df['위도 곡률값'] - point_list[i][0]) ** 2 + (df['경도 곡률값'] - point_list[i][1]) ** 2)

        name_list = ["거점"+str(i+1) for i in range(len(point_list))]
        df['선택'] = df.loc[:,  name_list].fillna(0).idxmin(axis=1)


    def cal_standard_curvature_value(self, df):
        df['위도간 거리(기준)_곡률값'] = min(df['위도\n(Latitude)']) * 3600
        df['경도간 거리(기준)_곡률값'] = min(df['경도\n(Longitude)']) * 3600

        df['위도간 거리(Y좌표 거리)'] = abs(df['위도\n(Latitude)'] * 3600 - df['위도간 거리(기준)_곡률값']) * 0.0245
        df['경도간 거리(X좌표 거리)'] = abs(df['경도\n(Longitude)'] * 3600 - df['경도간 거리(기준)_곡률값']) * 0.0306

        # 박스당 단가*물량 = VR
        df['VR'] = df['지역별 박스당 단가'] * df['Demand\n(Vi)']

        # .columns  박스당 단가* 물량*경도간 거리(X좌표 거리) = VRX
        df['VRX'] = df['지역별 박스당 단가'] * df['Demand\n(Vi)'] * df['경도간 거리(X좌표 거리)']

        # .columns  박스당 단가* 물량*위도간 거리(Y좌표 거리) = VRY
        df['VRY'] = df['지역별 박스당 단가'] * df['Demand\n(Vi)'] * df['위도간 거리(Y좌표 거리)']


    #apply함수를 위한 함수들
    #위도
    def func(self, dt) :
        for i in range(len(self.fixed_point)):
            if dt == self.name_list[i]:
                a  = self.fixed_point[i][0]/3600
                return a

    #경도
    def func1(self, dt) :
        for i in range(len(self.fixed_point)):
            if dt == self.name_list[i]:
                a = self.fixed_point[i][1]/3600
                return a

    def cal_point_result(self, df, point_list, d_count):

        s_df =pd.DataFrame()
        price_list = 150000000
        
        count = len(point_list)
        for i in range(len(point_list)):

            d_s = df[df['선택'] == '거점'+str(i+1)]
            # .(Y-Yi)^2 = 가상 센터에서 거래처 간의 거리
            VRX_Total = d_s['VRX'].sum()
            VRY_Total = d_s['VRY'].sum()
            VR_Total = d_s['VR'].sum()
            # VRX_SUM = d_3['VRX'].SUM()
            # VR_SUM = d_3['VR'].SUM()
            d_s['가상 거점_x'] = VRX_Total / VR_Total
            d_s['가상 거점_y'] = VRY_Total / VR_Total
            d_s['가상 거점_x(곡률값)'] = d_s['가상 거점_x'] / 0.0306 + d_s['경도간 거리(기준)_곡률값']
            d_s['가상 거점_y(곡률값)'] = d_s['가상 거점_y'] / 0.0245 + d_s['위도간 거리(기준)_곡률값']

            
           
            d_s['위경도 좌표_x'] = d_s['가상 거점_x(곡률값)']/3600
                
            d_s['위경도 좌표_y'] = d_s['가상 거점_y(곡률값)']/3600
            
            
            
            d_s['위경도 좌표_y'] = d_s['선택'].apply(self.func)
            d_s['위경도 좌표_x'] = d_s['선택'].apply(self.func1)
            

            tmp = d_s['위경도 좌표_x'].values.tolist()
            
            if len(tmp) > 0:
                if tmp[0] == None:
                    d_s['위경도 좌표_x'] = d_s['가상 거점_x(곡률값)']/3600
            if len(tmp) > 0:
                if tmp[0] == None:
                    d_s['위경도 좌표_y'] = d_s['가상 거점_y(곡률값)']/3600
            
            d_s['거래처수'] = d_s['선택'].count()

            # (Y-Yi)^2 = 가상 센터에서 거래처 간의 거리
            d_s['(Y-Yi)^2'] = (d_s['위도간 거리(Y좌표 거리)'] - d_s['가상 거점_y']) ** 2
            # (X-Xi)^2 = 가상 센터에서 거래처 간의 거리
            d_s['(X-Xi)^2'] = (d_s['경도간 거리(X좌표 거리)'] - d_s['가상 거점_x']) ** 2
            d_s['(Y-Yi)^2 + (X-Xi)^2'] = np.sqrt(d_s['(X-Xi)^2'] + d_s['(Y-Yi)^2'])
            d_s['X_곡률걊'] = d_s['가상 거점_x'] / 0.0306 + d_s['경도간 거리(기준)_곡률값']
            d_s['Y_곡률걊'] = d_s['가상 거점_y'] / 0.0245 + d_s['위도간 거리(기준)_곡률값']

            d_s['경도 좌표_X'] = d_s['X_곡률걊'] / 3600
            d_s['위도 좌표_Y'] = d_s['Y_곡률걊'] / 3600
            d_s['TC'] = d_s['VR'] * d_s['(Y-Yi)^2 + (X-Xi)^2']
            d_s['Transport Cost'] = d_s['TC'].sum()
            b = d_s['Demand\n(Vi)'].sum()
            d_s['총수량'] = int(b)

            s_df =pd.concat([s_df, d_s])

            d_s.to_csv("../result/총_" +str(len(point_list)) +"_거점별_데이터_"+str(i+1)+"번째 거점.csv", encoding='cp949')
            
            count-=1
            

        s_df.to_excel('../result/거점_'+ str(len(point_list))+'_시뮬레시션_raw(전주공장픽스).xlsx')

        s_df = s_df.loc[:, ['위경도 좌표_y', '위경도 좌표_x', '거래처수', '총수량', 'Transport Cost']]

        
        s_df = s_df.drop_duplicates()
        s_df = s_df.reset_index(drop=False)


        s_df['Warehous_Cost'] = price_list
        s_df['Total Cost'] = s_df['Transport Cost'] + s_df['Warehous_Cost'] * np.sqrt(1)

        return s_df

In [ ]:
# 조건.xlsx 파일 읽기

url = "조건.xlsx"
condition = files.upload()
condition_data = pd.DataFrame()
try:
    condition_data = pd.read_excel(io.BytesIO(condition[url]))
except:
    condition_data = pd.read_excel('/content/조건.xlsx')

Saving 조건.xlsx to 조건 (2).xlsx


In [ ]:
#조건 엑셀에서 위도 경도에 해당되는 데이터를 리스트화
latitude_list = condition_data['위도'].values.tolist()
longitude_list = condition_data['경도'].values.tolist()

# 동적 거점을 설정하기 위해서 거점 개수 만큼 0 개수 카운트 함
dynamic_count = latitude_list.count(0)

#fixed point 변수 = 곡률값 리스트 생성
points = [(i*3600, j*3600) for i, j in zip(latitude_list, longitude_list)]
fix_p = points[:len(points) - dynamic_count]

 

In [ ]:
url = "거점_raw.xlsx"



총 거점 = 동적 거점 + 고정 거점

결과 엑셀파일은 result폴더안에 생성됨

In [ ]:

# 결과가 저장될 폴더 생성
try:
  os.mkdir("/content/result")
except:
  pass

# RAW 데이터 불러오기
myfile = files.upload()

# 클래스 객체 선언
test_a = Cog(url, fix_p, dynamic_count)

# RAW 데이터 로드
data = pd.DataFrame()
try:
    data = pd.read_excel(io.BytesIO(myfile[url]))
except:
    data = pd.read_excel('/content/거점_raw.xlsx')

# 곡률값 계산 및 정렬
df = test_a.cal_curvature_value(data)

# 고정된 거점 곡률값과 동적 거점 곡률값 리스트 생성
test_list = test_a.make_position(df, dynamic_count)
print(test_list)

# 거점 클러스터링/분류
test_a.classified_position(df, test_list)

# 거점 이름 리스트
name_list = []
for i in range(len(test_list)):
    name_list.append('거점'+str(i+1))

#apply함수를 위한 함수들
#위도 선택 컬럼에서 거점에 따른 좌표 컬럼들 추가
def func_a(dt) :
    for i in range(len(test_list)):
        if dt == name_list[i]:
            return test_list[i][0]
        
#경도
def func1_a(dt) :
    for i in range(len(test_list)):
        if dt == name_list[i]:
            return test_list[i][1]


# 각 클러스터링 된 거점별로 선택거점 위도 경도 추가
df['선택거점_위도'] = df['선택'].apply(func_a)
df['선택거점_경도'] = df['선택'].apply(func1_a)

# VR / VRX / VRY 계산
test_a.cal_standard_curvature_value(df)


# 결과 산출 및 정리된 표 생성 warehouse_cost, total cost 등등 계산
#파일은 왼쪽 상단에 result 폴더안에 생성됨
result_df = test_a.cal_point_result(df, test_list, dynamic_count)

result_df.to_excel('/content/result/거점_'+ str(len(test_list)) +'_청주 시뮬레시션1(전주공장픽스).xlsx')

Saving 거점_raw.xlsx to 거점_raw (2).xlsx
[(135008.3574, 457654.69152), (135218.49120000002, 457084.6398), (135222.84107999998, 457067.94048), (135242.90639999998, 457118.3592)]
